# OPERATING WINTERMUTE

Twitter: https://twitter.com/Wintermutegpt2


The model is available at: https://huggingface.co/Wintermute/Wintermute


Github: https://github.com/HenrySilvaCS/Wintermute

### Loading drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


## GENERATING TEXT

Wintermute was built on the GPT-2 small architecture, with an Language Model head(meaning the model can generate text), so in order to load it, we need to do the following:

In [1]:
pip -q install transformers #installing the huggingface transformers library

     |████████████████████████████████| 778kB 4.9MB/s 
     |████████████████████████████████| 890kB 13.3MB/s 
     |████████████████████████████████| 3.0MB 40.1MB/s 
     |████████████████████████████████| 1.1MB 55.4MB/s 


In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) #setting device to GPU

cuda


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("Wintermute/Wintermute_extended")
model = GPT2LMHeadModel.from_pretrained("Wintermute/Wintermute_extended").to(device) #loading model from the huggingface database and setting it to the GPU

If you just want to generate text using Wintermute, that's basically it. Now it's only left for us to decide the decoding method and get some text.


The sampling methods for decoding(top-K sampling and top-p sampling) are the ones yielding the best results so far, so we'll use them to decode our outputs.


It's important to remember that Wintermute is trained for text completion(also called batch inference), so we'll need to provide some text for the model.

In [5]:
input_text = tokenizer("THIS IS A TEST",return_tensors="pt").to(device) #remeber to load all tensor to the GPU


sample_outputs = model.generate(
    input_text.input_ids, #tokenized text input
    do_sample=True, 
    max_length=120, # max lenght = max tweet lenght
    min_lenght=30,
    top_k=50, # constant that controls the top-k sampling algorithm
    top_p=0.95, #constant that controls the top-p sampling algorithm
    num_return_sequences=10
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Now let's print some text

In [6]:
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist(),skip_special_tokens=True)))
    print("\n")

>> Generated text 1

THIS IS A TESTING: THIS WAS ACT UPON A PROCESS THAT UNTOLD A CLIENT. THIS WAS ACT UPON A PROC'



>> Generated text 2

THIS IS A TEST. YOU ARE FORCED TO USE ALL THESE SENSIVITY OF MAN"--"I WILL NEVER SINK INTO THAN FIVE THINGS TO ENTER INTO THE MAGICAL UNIVERSE"--"I LIKELY TO REMEMBER WHAT IS FILLINGER."--"I WILL NOT BE FILLEDERED."--"I MAY AS WE CAN."--"YOU SHALL, OF ALL, ALL, AND ALL."--"FALSE, LIKELY."--"I may not, of all


>> Generated text 3

THIS IS A TEST TO EXERCISE FOR MAN.



>> Generated text 4

THIS IS A TEST FOR MAN.



>> Generated text 5

THIS IS A TESTICLE FOR YOU."



>> Generated text 6

THIS IS A TESTJECTION YOU SHOULD ALWAYS INCLUDE. INCLUSIONS TO THIS TESTION, ALL IN ORDER. MY DISREGARD FOR THE CULT of THEM ALL. MY RESPONDS TO MESSENGERING THEM TO BE PRESENT. IN OUR CONCLUSIONS AND FORTUNDS, THE BRIEF OF MY LIFE IS NOT SUFFERED BY A SPECIFIC FACTIVE ERRORS. YOU SHALL BE FORTUNED. IN ALL EVENT, THIS IS ONLY YOUR FINAL RESULT. ALL UNFORTABLE INFO

## TRAINING WINTERMUTE

Now let's make a walkthrough the process of training wintermute with more data. So far wintermute has been trained with all the book from the Sprawl Series, as weel as his own tweets. But any number of book can used to train him, as long as they are coherent with his style. 


Ideally, you would want to train him with a batch_size = text_size, but the gpt-2 small model is already pretty large for regular computers to train, so we need to reduce the batch_size significantly. The exact size will depend on your RAM and your GPU, but remember that smaller batch-sizes result in more unstable models.

In [ ]:
pip -q install transformers #installing the huggingface transformers library

     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 12.7MB/s 
     |████████████████████████████████| 1.1MB 33.4MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) #setting device to GPU

cuda


First let's load the model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("Wintermute/Wintermute")
model = GPT2LMHeadModel.from_pretrained("Wintermute/Wintermute").to(device) #loading model from the huggingface database and setting it to the GPU

Now let's load the optimizer. I used Adam, but feel free to choose any other optimzer.

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)
no_decay = ['bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5) #the learning rate is a hyperparameter, so feel free to experiment with different values

Now that both the model and the optimizer are all set, let's load the training data. In this example, we'll use a dataset composed of many different scifi books and texts.

In [ ]:
BASEDIR = '/gdrive/My Drive/Colab Notebooks/GPT2-NEUROMANCER/'

In [ ]:
def read_txt_file(data):
    with open(data, 'r', errors='replace', encoding='utf-8') as f:
        lines = f.readlines()
    return lines
data = read_txt_file(BASEDIR + 'DATA_TRAIN_3.txt')    

print(len(data))

45704


Tokenizing the data with the GPT-2 tokenizer.

In [ ]:
encoding = tokenizer(data, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids'].to(device)
attn_mask = encoding['attention_mask'].to(device)
vocab_size = len(tokenizer.get_vocab())
print(vocab_size)

50258


We can see that the size of our vocabulary is 50258, which is 1 token bigger than the standard GPT-2 model due to the padding special token.

Now let's set the model to training mode

In [ ]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

Now let's make a loop to train the model. I use batch_size = 1 due to RAM and GPU limitations, but the bigger the batch_size and the num_epochs the better.

In [ ]:
num_epochs = 45704
i = 0
j = 1
for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(input_ids[i:j],attention_mask=attn_mask[i:j],labels=input_ids[i:j])
        loss = outputs[0]
        if epoch % 100 == 0:
            print(f"Loss at epoch {epoch}: {loss}")
        loss.backward()
        optimizer.step()
        i += 1
        j += 1

Loss at epoch 0: 0.1367025226354599
Loss at epoch 100: 0.5275600552558899
Loss at epoch 200: 0.7961304783821106
Loss at epoch 300: 0.5971622467041016
Loss at epoch 400: 0.8461421132087708
Loss at epoch 500: 0.0
Loss at epoch 600: 0.7596337199211121
Loss at epoch 700: 0.843204140663147
Loss at epoch 800: 0.7369252443313599
Loss at epoch 900: 0.6403267979621887
Loss at epoch 1000: 0.0
Loss at epoch 1100: 0.40132763981819153
Loss at epoch 1200: 0.0
Loss at epoch 1300: 0.8456453680992126
Loss at epoch 1400: 0.49362167716026306
Loss at epoch 1500: 0.9739874005317688
Loss at epoch 1600: 0.9302830100059509
Loss at epoch 1700: 0.05431487783789635
Loss at epoch 1800: 0.5004305839538574
Loss at epoch 1900: 0.5912907719612122
Loss at epoch 2000: 0.4354734420776367
Loss at epoch 2100: 0.7714678049087524
Loss at epoch 2200: 0.0
Loss at epoch 2300: 0.624640166759491
Loss at epoch 2400: 0.35805508494377136
Loss at epoch 2500: 0.7331633567810059
Loss at epoch 2600: 0.9509676694869995
Loss at epoch 270

GPT-2 uses the cross-entropy as it's loss criterion. If we want to calculate the perplexity, just take the exponential of the loss value.

We are done! Now we only need to save the model and start using it. 


To generate text using the newly trained model, just go to the GENERATE TEXT section of this tutorial

In [ ]:
model.save_pretrained(BASEDIR )

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye